# MaiChart AI - Colab Training

1. **Check GPU**: Runtime -> Change runtime type -> T4 GPU.

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/lvyirgil/maimai-chat-ai.git /content/maimai-chat-ai
%cd /content/maimai-chat-ai

In [ ]:
!pip install -r requirements.txt

In [ ]:
!python -m src.data.preprocess

In [ ]:
!python train_gpu.py --config configs/default.yaml